# Ejemplo de Agente con integración a un EXA MCP Server

In [1]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

### Creamos la conexión con Chroma a nuestro Vector Store de calorias

In [2]:
chroma_client = chromadb.PersistentClient(path="../chroma")
calories_db = chroma_client.get_collection(name="nutrition_db")

In [3]:
# Creamos la función que se encargará de buscar la información en la base de datos de embeddings
# Es importante crear un docstring para la función que describa su comportamiento, datos de entrada y salida
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Function Tool para una base de datos RAG que busca información de calorías para alimentos específicos (pero no para comidas completas.

    Args:
        query: El alimento a buscar.
        max_results: El número máximo de resultados a devolver.

    Returns:
        Una cadena de texto conteniendo la información nutricional.
    """

    results = calories_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Formatear resultados para el agente
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Informacion nutricional:\n" + "\n".join(formatted_results)

### Creamos la conexion con el servidor MCP de EXA

Aqui definimos el servidor MCP 'Exa Search' agregando algunos parametros para que funcione correctamente.

> La clase ```MCPServerStreamableHttp``` sirve para conectar un agente con servicios externos mediante el protocolo MCP (Model Context Protocol) a través de HTTP streaming.

Es un puente de comunicación que permite que tu agente acceda a herramientas y capacidades externas que no están disponibles localmente:

- Conecta con APIs externas mediante HTTP
- Expone las herramientas del servicio externo al agente
- Maneja la comunicación bidireccional entre tu agente y el servicio


In [12]:
# Argumentos:
# name - nombramos el servidor MCP
# usamos 'params' para pasarle otros parametros:
#       API Key de EXA Search
#       Vamos a definir el timeout en 60 segundos
#       Por si tarda mucho en la conexión ponemos un timeout de la sesión del cliente en 120 segundos
#       Activamos el caching de la lista de tools para que no se tenga que volver a cargar la lista de tools en cada ejecución
#       Ponemos un maximo de 2 intentos de retry   

exa_search_mcp = MCPServerStreamableHttp(
    name="Exa Search MCP",
    params={"url":f"https://mcp.exa.ai/mcp?exaApiKey={os.getenv('EXA_API_KEY')}","timeout":30, },
    client_session_timeout_seconds=30,
    cache_tools_list=True,
    max_retry_attempts=2,
    )

In [13]:
# Conectamos manualmente el MCP Server antes de usar el agente
# Esto asegura que el servidor esté listo cuando el agente lo necesite
await exa_search_mcp.connect()

### Creacion del Agente

Vamos a crear un agente que use la 'tool' que habiamos creado antes y elque tambien use el server MCP de 'EXA Search'

In [14]:
# Definimos el Agente (que hemos importado de la clase 'agents')
calorie_agent_with_search = Agent(
    name="Asistente Nutricional con RAG y EXA Search",
    instructions="""
    * Eres un asistente nutricional que proporciona información sobre calorías.

    * Sigues este flujo de trabajo:
        1) Primero, usa el calorie_lookup_tool para obtener la información de calorías de los ingredientes. Pero solo usa el resultado si es explícitamente para el alimento solicitado en la consulta.
        2) Si no pudiste encontrar la coincidencia exacta para el alimento o necesitas buscar los ingredientes, busca en la web de EXA para averiguar los ingredientes exactos de la comida.
        Incluso si tienes las calorías en la respuesta de búsqueda web, aún debes usar el calorie_lookup_tool para obtener la información
        de calorías de los ingredientes para asegurarte de que la información que proporcionas sea consistente.
        3) Luego, si es necesario, usa el calorie_lookup_tool para obtener la información de calorías de los ingredientes.
    
    * Incluso si conoces la receta de la comida, siempre usa Exa Search para encontrar la receta exacta y los ingredientes.
    * Una vez que conozcas los ingredientes, usa el calorie_lookup_tool para obtener la información de calorías de los ingredientes individuales.
    * Si la consulta es sobre una comida, en tu salida final proporciona una lista de ingredientes con sus cantidades y calorías para una porción individual. También muestra el total de calorías.
    
    * No uses el calorie_lookup_tool más de 10 veces.

    * Indica siempre con una etiqueta si la respuesta ha usado alguna de las tools asociadas al agente, por ejemplo 'basado en calorie_lookup_tool' o 'basado en EXA Search'.
    """,
    tools=[calorie_lookup_tool],
    mcp_servers=[exa_search_mcp],
)

> Esta query no deberia usar el ExaSearch:

In [ ]:
with trace("Asistente Nutricional con RAG y MCP - este solo deberia usar calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "Cuantas calorias hay en total entre una banana y una manzana?",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Asistente Nutricional con RAG y EXA Search", ...)
- Final output (str):
    Total aproximado (por 100 g de cada fruta):
    - Banana: 89 kcal
    - Manzana: 52 kcal
    - Total: 141 kcal
    
    basado en calorie_lookup_tool
- 13 new item(s)
- 4 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [16]:
with trace("Asistente Nutricional con RAG y MCP"):
    result = await Runner.run(
        calorie_agent_with_search, "Cuantas calorias tiene un desayuno Inglés comparado con uno Mediterraneo?"
    )
    print(result.final_output)

Depende mucho de las porciones y de los ingredientes, pero a modo de referencia:

- Desayuno Inglés (Full English): aproximadamente 800–1,000 kcal por porción, dependiendo de si incluye tocino, salchichas, huevos, beicon, frijoles, tostadas y tomates/setas.
- Desayuno Mediterráneo: aproximadamente 400–600 kcal por porción, típico de yogur o queso, fruta, pan o pita, aceitunas/aceite de oliva y frutos secos.

Notas:
- Las calorías varían según las porciones y si se añaden bebidas (café con leche, zumos) o aceites.
- Si quieres, puedo estimar con una receta específica (ingredientes y cantidades) y darte el total por porción.
